Tendências de Consumo de Combustíveis por tipo e por ano

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, sum, col

spark = SparkSession.builder \
    .appName("ANP Combustíveis Líquidos") \
    .getOrCreate()

In [ ]:
df_vendas_atual = spark.read.parquet("../../datalake/silver/Liquidos_Vendas_Atual")

In [ ]:
df_vendas_atual = df_vendas_atual.withColumn(
    "quantidade_de_produto_(mil_m3)",
    regexp_replace(col("quantidade_de_produto_(mil_m3)"), ",", ".").cast("double")
)
df_agrupado = df_vendas_atual.groupBy("ano", "nome_do_produto") \
    .agg(sum("quantidade_de_produto_(mil_m3)").alias("volume_total_(mil_m3)")) \
    .orderBy("ano")

df_agrupado.show()

+----+----------------+---------------------+
| ano| nome_do_produto|volume_total_(mil_m3)|
+----+----------------+---------------------+
|2017|        Diesel B|   54759.359812000635|
|2017|      Gasolina C|   44156.214131000306|
|2017|Etanol Hidratado|    13657.16730400005|
|2017|      Óleo Comb.|   3380.8778530000004|
|2018|      Gasolina C|   38384.320629000285|
|2018|Etanol Hidratado|    19400.52250100006|
|2018|      Óleo Comb.|          2312.568761|
|2018|        Diesel B|    55652.50517000063|
|2019|      Gasolina C|    38166.35208400028|
|2019|      Óleo Comb.|   1890.6110630000003|
|2019|        Diesel B|   57304.043884000566|
|2019|Etanol Hidratado|   22545.786087000088|
|2020|      Gasolina C|     35823.4258580003|
|2020|      Óleo Comb.|          2019.188482|
|2020|        Diesel B|    57469.00485300072|
|2020|Etanol Hidratado|    19258.63851300007|
|2021|Etanol Hidratado|    16792.04809300005|
|2021|        Diesel B|    62091.53277900068|
|2021|      Gasolina C|   39315.58

In [7]:
df_agrupado.write.mode("overwrite").parquet("../../datalake/gold/consumo_combustiveis_ano")